<a href="https://colab.research.google.com/github/tchintchie/HR_Analysis/blob/master/HR_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HR Analysis

The HR department has tasked you with building a prediction model.
You are supposed to figure out if an employee is going to leave or no.

Below are two files, one for training and the other for testing.
The data is nicely formatted, with no missing values, no outliers... can be used as-is but you can wrangle it if you like.

We are predicting the 'left' column, 1 meaning yes and 0 meaning no.

The purpose of this exercise is to get you comfortable with using different models, reading the documentation, trying stuff out...

Here are the steps:
1. Load the HR_train data and do EDA as you see fit. I am not giving instructions as you should explore and decide what to do by yourself.
2. Split data into training/testing sets: https://pastebin.com/1pJkrTCV
3. Build your model with the train data: https://pastebin.com/4JqpvWxV
4. With your model built you need to import the testing data (HR_test) and try your model on it: https://pastebin.com/kpP3vTGf
5. Once done and happy with your score you need to summarize your findings: the HR department would like to know which groups are the most risky and if you see any trends or have suggestions for them on how to improve retention. (for that you will need to do EDA, dataviz... but as mentioned at the beginning, that's up to you).

And that's it. 
Try to raise the score by experimenting with column selection, filtering, model tweaking and the introduction of new models.
e.g. if you want to try a decision tree you just do: https://pastebin.com/ffVM0Sit

There are many more models if you want to play around: https://scikit-learn.org/stable/supervised_learning.html
They all follow the same logic of:
1. Import
2. Create an instance of it
3. Train
4. Score

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [8]:
df = pd.read_csv("HR_train.csv")
df1 = pd.read_csv("HR_test.csv")
df = pd.concat([df, df1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
df.head()

,Department,Work_accident,average_montly_hours,department,last_evaluation,left,number_project,promotion_last_5years,salary,satisfaction_level,time_spend_company,work_accident
0,sales,0.0,157,NaN,0.53,1,2,0,low,0.38,3,NaN
1,sales,0.0,262,NaN,0.86,1,5,0,medium,0.80,6,NaN
2,sales,0.0,272,NaN,0.88,1,7,0,medium,0.11,4,NaN
3,sales,0.0,223,NaN,0.87,1,5,0,low,0.72,5,NaN
4,sales,0.0,159,NaN,0.52,1,2,0,low,0.37,3,NaN


In [10]:
df.shape

(14999, 12)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14999 entries, 0 to 499
Data columns (total 12 columns):
Department               14499 non-null object
Work_accident            14499 non-null float64
average_montly_hours     14999 non-null int64
department               500 non-null object
last_evaluation          14999 non-null float64
left                     14999 non-null int64
number_project           14999 non-null int64
promotion_last_5years    14999 non-null int64
salary                   14999 non-null object
satisfaction_level       14999 non-null float64
time_spend_company       14999 non-null int64
work_accident            500 non-null float64
dtypes: float64(4), int64(5), object(3)
memory usage: 1.5+ MB
